# Trying Auto Correct

In [ ]:
!pip install autocorrect

In [ ]:
from autocorrect import spell
print(spell('caaaar'))
print(spell(u'mussage'))
print (spell(u'survice'))
print (spell(u'hte'))

# Loading Required Libraries

In [1]:
##Loading required libraries
import numpy as np
import pandas as pd
import pickle
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
C:\Users\M1030574\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\M1030574\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\M1030574\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood 

# Data Loading and processing

In [2]:
path='C:\\Users\\M1030574\\Desktop\\NLP COE\\Text Classification\\Course Era Classification\\'
filename='reviews.csv'

In [ ]:
data=pd.read_csv(path+filename)
##Groupin 5 labels to 3 labels as positive neutral and negative
data['Label']=data['Label'].replace({1:1,2:1,3:2,4:3,5:3})
label1=data[data['Label']==1].sample(n=700,random_state=1234)
label2=data[data['Label']==2].sample(n=500,random_state=1234)
label3=data[data['Label']==3].sample(n=1300,random_state=1234)
data1=pd.concat([label1,label2,label3])
##Renaming label codes to 0,1,2 for label names 
data1['Label']=data['Label'].replace({1:0,2:1,3:2})

In [ ]:
# 1,2>negative-1
# 3>neutral-2
# 4,5>positive-3

In [ ]:
np.unique(data1['Label'])

In [ ]:
##Summarizing data
data1.groupby("Label").agg({'Id':'count'})

In [ ]:
data1['Review']=data1['Review'].str.lower()

In [ ]:
# ##Converting it into codes 
# data1['Label']= pd.Categorical(data1['Label'])
# data1['Label_Code']=data1['Label'].cat.codes

In [ ]:
x_train, x_test, y_train,y_test = train_test_split(data1[['Id','Review']],data1[['Label']], stratify=data1['Label'],
                                                   test_size=0.3, random_state=42)
Train=pd.concat([x_train['Review'],y_train['Label']],axis=1)
Test=pd.concat([x_test['Review'],y_test['Label']],axis=1)

# Writing Files 

In [ ]:
data1.to_csv(path+"Processed_Data.csv")
Train.to_csv(path+"Train_Data.csv")
Test.to_csv(path+"Test_Data.csv")

In [ ]:
Train=pd.read_csv(path+"Train_Data.csv")

In [ ]:
Train=Train[['Review','Label']]

# Model Training

In [ ]:

# Optional model configuration
model_args = ClassificationArgs(num_train_epochs=5,overwrite_output_dir=True)

print("model_args")


# bert:Bert,bert-base-uncased

# xlnet:xlnet,xlnet-base-cased  
#'roberta','roberta-base',
#electra-base,google/electra-base-discriminator"
#"google/electra-small-discriminator,electra

# Create a ClassificationModel
model_bert = ClassificationModel(
    model_type='roberta',
    model_name="roberta-base",
    num_labels=3,
    args=model_args,
#     use_cuda=False
)



print("model_traing")
# Train the model
model_bert.train_model(Train)



In [ ]:
Model_Name='electra_small_3.sav'

In [ ]:
pickle.dump(model_bert, open(path+Model_Name,'wb'))

# Checking Model Accuracy

In [3]:
##Loading Model and checking accuracy
Model_Name='electra_small_3'
model=pickle.load(open(path+Model_Name+'.sav','rb'))

In [4]:
##Accuracy for train data 
Train_Predictions=model.predict(Train['Review'].tolist())
Train_Prediction_df=pd.DataFrame(Train_Predictions[0])
print(accuracy_score(Train[['Label']],Train_Prediction_df))

NameError: name 'Train' is not defined

In [ ]:
##Final DataFrame for traindata to check accuracy
Train_Op=pd.concat([Train[['Review','Label']].reset_index(drop=True),Train_Prediction_df.reset_index(drop=True)],axis=1)
Train_Op.columns=['Review','Actual_Label','Predicted_Label']

In [ ]:
Train_Op.to_csv(path+'Train_Op'+Model_Name+'.csv')

In [ ]:
Test=pd.read_csv(path+"Test_Data.csv")

In [ ]:
Test=Test[['Review','Label']]

In [ ]:
##Accuracy for test data 
Test_Predictions=model.predict(Test['Review'].tolist())
Test_Prediction_df=pd.DataFrame(Test_Predictions[0])
print(accuracy_score(Test[['Label']],Test_Prediction_df))

In [ ]:
##Final DataFrame for testdata to check accuracy
Test_Op=pd.concat([Test.reset_index(drop=True),Test_Prediction_df.reset_index(drop=True)],axis=1)
Test_Op.columns=['Review','Actual_Label','Predicted_Label']

In [ ]:
Test_Op.to_csv(path+'Test_Op'+Model_Name+'.csv')

In [ ]:
##lABEL WISE CHECK FOR IT
##Reading Train Output
Train_op=pd.read_csv(path+'Train_Op'+Model_Name+'.csv')
Train_op=Train_op[['Review','Actual_Label','Predicted_Label']]
Train_op_Label0=Train_op[Train_op['Actual_Label']==0]
Train_op_Label1=Train_op[Train_op['Actual_Label']==1]
Train_op_Label2=Train_op[Train_op['Actual_Label']==2]
print('TrainLabel_0='+str(accuracy_score(Train_op_Label0['Actual_Label'],Train_op_Label0['Predicted_Label'])))
print('TrainLabel_1='+str(accuracy_score(Train_op_Label1['Actual_Label'],Train_op_Label1['Predicted_Label'])))
print('TrainLabel_2='+str(accuracy_score(Train_op_Label2['Actual_Label'],Train_op_Label2['Predicted_Label'])))

In [ ]:
##lABEL WISE CHECK FOR IT
##Reading Test Output
Test_op=pd.read_csv(path+'Test_Op'+Model_Name+'.csv')
Test_op=Test_op[['Review','Actual_Label','Predicted_Label']]
Test_op_Label0=Test_op[Test_op['Actual_Label']==0]
Test_op_Label1=Test_op[Test_op['Actual_Label']==1]
Test_op_Label2=Test_op[Test_op['Actual_Label']==2]
print('TestLabel_0='+str(accuracy_score(Test_op_Label0['Actual_Label'],Test_op_Label0['Predicted_Label'])))
print('TestLabel_1='+str(accuracy_score(Test_op_Label1['Actual_Label'],Test_op_Label1['Predicted_Label'])))
print('TestLabel_2='+str(accuracy_score(Test_op_Label2['Actual_Label'],Test_op_Label2['Predicted_Label'])))

# Loading Model and predicting

In [56]:
##Loading Model and checking accuracy
path='C:\\Users\\M1030574\\Desktop\\NLP COE\\Text Classification\\Course Era Classification\\'
Model_Name='electra_small_3'
model=pickle.load(open(path+Model_Name+'.sav','rb'))

In [57]:
text=["i ve already written a review for part 1 and i have the same opinion about this one. the course is rather poor and not challenging. only general information about relevant topics that as well read on wikipedia. no exercises, no code assignments. a lot of this content was repeated from first two parts of this specialization."]
predicted=model.predict(text)[0][0]

In [58]:
label = {0 : "Negative", 1 : "Neutral",2:"Positive"}

In [62]:
for key,value in label.items():
    if key == predicted:
         print(text,value)
       

['i ve already written a review for part 1 and i have the same opinion about this one. the course is rather poor and not challenging. only general information about relevant topics that as well read on wikipedia. no exercises, no code assignments. a lot of this content was repeated from first two parts of this specialization.'] Negative


# Model evaluation for negative feedbacks 

# BERT

In [ ]:
##Bert Model
import pandas as pd
##Reasing Train and Test Bert Output 
Train_op_bert=pd.read_csv("C:\\Users\\M1030574\\Desktop\\NLP COE\\Text Classification\\Course Era Classification\\Train_Opbert.csv")
Test_op_bert=pd.read_csv("C:\\Users\\M1030574\\Desktop\\NLP COE\\Text Classification\\Course Era Classification\\Test_Opbert.csv")

In [ ]:
Train_op_bert.head(2)

In [ ]:
Train_op_bert_negative=Train_op_bert[Train_op_bert['Actual_Label']==0]
Test_op_bert_negative=Test_op_bert[Test_op_bert['Actual_Label']==0]

In [ ]:
Train_op_bert_negative.groupby("Predicted_Label").agg({'Review':'count'})

In [ ]:
Test_op_bert_negative.groupby("Predicted_Label").agg({'Review':'count'})

# Roberta

In [ ]:
##Bert Model
import pandas as pd
##Reasing Train and Test Bert Output 
Train_op_Roberta=pd.read_csv("C:\\Users\\M1030574\\Desktop\\NLP COE\\Text Classification\\Course Era Classification\\Train_Oproberta.csv")
Test_op_Roberta=pd.read_csv("C:\\Users\\M1030574\\Desktop\\NLP COE\\Text Classification\\Course Era Classification\\Test_Oproberta.csv")

In [ ]:
Train_op_Roberta_negative=Train_op_Roberta[Train_op_Roberta['Actual_Label']==0]
Test_op_Roberta_negative=Test_op_Roberta[Test_op_Roberta['Actual_Label']==0]

In [ ]:
Train_op_Roberta_negative.groupby("Predicted_Label").agg({'Review':'count'})

In [ ]:
Test_op_Roberta_negative.groupby("Predicted_Label").agg({'Review':'count'})

# XLNET

In [ ]:
##Bert Model
import pandas as pd
##Reasing Train and Test Bert Output 
Train_op_xlnet=pd.read_csv("C:\\Users\\M1030574\\Desktop\\NLP COE\\Text Classification\\Course Era Classification\\Train_Opxlnet.csv")
Test_op_xlnet=pd.read_csv("C:\\Users\\M1030574\\Desktop\\NLP COE\\Text Classification\\Course Era Classification\\Test_Opxlnet.csv")

In [ ]:
Train_op_xlnet_negative=Train_op_xlnet[Train_op_xlnet['Actual_Label']==0]
Test_op_xlnet_negative=Test_op_xlnet[Test_op_xlnet['Actual_Label']==0]

In [ ]:
Train_op_xlnet_negative.groupby("Predicted_Label").agg({'Review':'count'})

In [ ]:
Test_op_xlnet_negative.groupby("Predicted_Label").agg({'Review':'count'})

# Ensembling Model

# Voting

In [ ]:
#Concatenating model Outouts

In [ ]:
Train_Op_Bert=pd.read_csv("C:\\Users\\M1030574\\Desktop\\NLP COE\\Text Classification\\Course Era Classification\\Train_Opbert.csv")
Train_Op_Roberta=pd.read_csv("C:\\Users\\M1030574\\Desktop\\NLP COE\\Text Classification\\Course Era Classification\\Train_Oproberta.csv")
Train_Op_Xlnet=pd.read_csv("C:\\Users\\M1030574\\Desktop\\NLP COE\\Text Classification\\Course Era Classification\\Train_Opxlnet.csv")

In [ ]:
Train_Op_Bert=Train_Op_Bert[['Review','Actual_Label','Predicted_Label']]
Train_Op_Bert.columns=['Review','Actual_Label','Predicted_Label_Bert']

In [ ]:
Train_Op_Roberta=Train_Op_Roberta[['Review','Predicted_Label']]
Train_Op_Roberta.columns=['Review_Roberta','Predicted_Label_Roberta']

In [ ]:
Train_Op_Xlnet=Train_Op_Xlnet[['Review','Predicted_Label']]
Train_Op_Xlnet.columns=['Review_Xlnet','Predicted_Label_Xlnet']

In [ ]:
Train_Concatenated_Op=pd.concat([Train_Op_Bert,Train_Op_Roberta,Train_Op_Xlnet],axis=1)

In [ ]:
Train_Concatenated_Op.columns

In [ ]:
Train_Concatenated_Op=Train_Concatenated_Op[['Review','Actual_Label','Predicted_Label_Bert','Predicted_Label_Roberta','Predicted_Label_Xlnet']]

In [ ]:
Train_Concatenated_Op.to_excel('C:\\Users\\M1030574\\Desktop\\NLP COE\\Text Classification\\Course Era Classification\\Train_Op_Count.xlsx')

In [ ]:
##Doing Same for Test data 
Test_Op_Bert=pd.read_csv("C:\\Users\\M1030574\\Desktop\\NLP COE\\Text Classification\\Course Era Classification\\Test_Opbert.csv")
Test_Op_Roberta=pd.read_csv("C:\\Users\\M1030574\\Desktop\\NLP COE\\Text Classification\\Course Era Classification\\Test_Oproberta.csv")
Test_Op_Xlnet=pd.read_csv("C:\\Users\\M1030574\\Desktop\\NLP COE\\Text Classification\\Course Era Classification\\Test_Opxlnet.csv")

In [ ]:
Test_Op_Bert=Test_Op_Bert[['Review','Actual_Label','Predicted_Label']]
Test_Op_Bert.columns=['Review','Actual_Label','Predicted_Label_Bert']

In [ ]:
Test_Op_Roberta=Test_Op_Roberta[['Review','Predicted_Label']]
Test_Op_Roberta.columns=['Review_Roberta','Predicted_Label_Roberta']

In [ ]:
Test_Op_Xlnet=Test_Op_Xlnet[['Review','Predicted_Label']]
Test_Op_Xlnet.columns=['Review_Xlnet','Predicted_Label_Xlnet']

In [ ]:
Test_Concatenated_Op=pd.concat([Test_Op_Bert,Test_Op_Roberta,Test_Op_Xlnet],axis=1)

In [ ]:
Test_Concatenated_Op=Test_Concatenated_Op[['Review','Actual_Label','Predicted_Label_Bert','Predicted_Label_Roberta','Predicted_Label_Xlnet']]

In [ ]:
Test_Concatenated_Op.to_excel('C:\\Users\\M1030574\\Desktop\\NLP COE\\Text Classification\\Course Era Classification\\Test_Op_Count.xlsx')

In [ ]:
Test_Concatenated_Op['Probality_label0']=pd.concat([Test_Concatenated_Op.Predicted_Label_Bert==0,Test_Concatenated_Op.Predicted_Label_Roberta==0,Test_Concatenated_Op.Predicted_Label_Xlnet==0],axis=1).sum(axis=1)
Test_Concatenated_Op['Probality_label1']=pd.concat([Test_Concatenated_Op.Predicted_Label_Bert==1,Test_Concatenated_Op.Predicted_Label_Roberta==1,Test_Concatenated_Op.Predicted_Label_Xlnet==1],axis=1).sum(axis=1)
Test_Concatenated_Op['Probality_label2']=pd.concat([Test_Concatenated_Op.Predicted_Label_Bert==2,Test_Concatenated_Op.Predicted_Label_Roberta==2,Test_Concatenated_Op.Predicted_Label_Xlnet==2],axis=1).sum(axis=1)


In [ ]:
Test_Concatenated_Op['Probality_label0']=Test_Concatenated_Op['Probality_label0']/3
Test_Concatenated_Op['Probality_label1']=Test_Concatenated_Op['Probality_label1']/3
Test_Concatenated_Op['Probality_label2']=Test_Concatenated_Op['Probality_label2']/3

In [ ]:
Test_Concatenated_Op['Max_Probality']=Test_Concatenated_Op[["Probality_label0", "Probality_label1","Probality_label2"]].max(axis=1)

In [ ]:
Test_Concatenated_Op['Predicted_Label']=4

In [ ]:
Test_Concatenated_Op['Predicted_Label'].loc[Test_Concatenated_Op['Max_Probality']==Test_Concatenated_Op['Probality_label0']] = 0
Test_Concatenated_Op['Predicted_Label'].loc[Test_Concatenated_Op['Max_Probality']==Test_Concatenated_Op['Probality_label1']] = 1
Test_Concatenated_Op['Predicted_Label'].loc[Test_Concatenated_Op['Max_Probality']==Test_Concatenated_Op['Probality_label2']] = 2

In [ ]:
np.unique(Test_Concatenated_Op['Predicted_Label'])

In [ ]:
Test_Concatenated_Op['Accuracy']=0

In [ ]:
Test_Concatenated_Op['Accuracy'].loc[Test_Concatenated_Op['Predicted_Label']==Test_Concatenated_Op['Actual_Label']]=1

In [ ]:
np.unique(Test_Concatenated_Op['Accuracy'])

In [ ]:
np.sum(Test_Concatenated_Op['Accuracy'])/np.count(Test_Concatenated_Op['Accuracy'])